**Get the training dataset**

In [35]:
!wget -O RNN-HAR-2D-Pose-database.zip https://drive.google.com/u/1/uc?id=1IuZlyNjg6DMQE3iaO1Px6h1yLKgatynt

--2021-08-05 16:34:23--  https://drive.google.com/u/1/uc?id=1IuZlyNjg6DMQE3iaO1Px6h1yLKgatynt
Resolving drive.google.com (drive.google.com)... 172.217.161.174, 2404:6800:4005:80f::200e
Connecting to drive.google.com (drive.google.com)|172.217.161.174|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-00-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jn6s6e63pc0q2b2slonah7ld1lij0voe/1628156025000/01198563878503023152/*/1IuZlyNjg6DMQE3iaO1Px6h1yLKgatynt [following]
--2021-08-05 16:34:25--  https://doc-10-00-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jn6s6e63pc0q2b2slonah7ld1lij0voe/1628156025000/01198563878503023152/*/1IuZlyNjg6DMQE3iaO1Px6h1yLKgatynt
Resolving doc-10-00-docs.googleusercontent.com (doc-10-00-docs.googleusercontent.com)... 142.250.66.33, 2404:6800:4005:81c::2001
Connecting to doc-10-00-docs.googleusercontent.com (doc-10-00-docs.googleusercontent.com)|142.250.66

In [36]:
!unzip RNN-HAR-2D-Pose-database.zip

Archive:  RNN-HAR-2D-Pose-database.zip
replace RNN-HAR-2D-Pose-database/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


**List first two rows of training set**

In [37]:
!head -2 RNN-HAR-2D-Pose-database/X_train.txt

295.914,161.579,307.693,203.413,281.546,203.368,274.997,251.562,267.194,293.253,337.619,204.669,347.958,255.443,341.541,295.866,286.81,289.393,297.196,355.832,297.22,405.371,321.967,291.959,327.143,358.408,328.528,411.922,294.546,156.42,305.002,156.418,0,0,318.083,161.632
295.855,161.6,307.684,203.408,281.529,203.385,274.989,251.574,267.191,291.961,337.615,204.646,347.974,254.209,344.093,295.816,286.803,289.377,297.165,355.827,297.205,404.095,323.248,290.652,324.564,358.409,328.493,410.63,293.252,157.686,303.706,157.706,0,0,318.024,161.654


**Clone the codebase**

In [38]:
!git clone https://github.com/spmallick/learnopencv.git

Cloning into 'learnopencv'...
remote: Enumerating objects: 9208, done.
^Cceiving objects:  16% (1492/9208), 10.29 MiB | 2.20 MiB/s   


In [39]:
%cd learnopencv/Human-Action-Recognition-Using-Detectron2-And-Lstm

[Errno 2] No such file or directory: 'learnopencv/Human-Action-Recognition-Using-Detectron2-And-Lstm'
/media/sunt/DATA/AI-PRACTICE/Human-Action-Recognition-Using-Detectron2-And-Lstm


**Install dependencies**

In [40]:
!pip install -r requirements.txt

In [41]:
DATASET_PATH = "RNN-HAR-2D-Pose-database/"

In [42]:
from argparse import ArgumentParser

def configuration_parser(parent_parser):
    parser = ArgumentParser(parents=[parent_parser], add_help=False)
    parser.add_argument('--batch_size', type=int, default=512)
    parser.add_argument('--epochs', type=int, default=400)
    parser.add_argument('--data_root', type=str, default=DATASET_PATH)
    parser.add_argument('--learning_rate', type=float, default=0.0001)
    parser.add_argument('--num_class', type=int, default=6)
    return parser

In [43]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor

from src.lstm import ActionClassificationLSTM, PoseDataModule

In [45]:
def do_training_validation():
    pl.seed_everything(21)    
    parser = ArgumentParser()
    parser = pl.Trainer.add_argparse_args(parser)
    parser = configuration_parser(parser)
    # args = parser.parse_args()
    args, unknown = parser.parse_known_args()
    # init model    
    hidden_dim = 50
    WINDOW_SIZE = 32
    model = ActionClassificationLSTM(WINDOW_SIZE, hidden_dim, learning_rate=args.learning_rate)
    data_module = PoseDataModule(data_root=args.data_root,
                                        batch_size=args.batch_size)    
    #save only the top 1 model based on val_loss
    checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor='val_loss')
    lr_monitor = LearningRateMonitor(logging_interval='step')  
    #trainer
    trainer = pl.Trainer.from_argparse_args(args,
        # fast_dev_run=True,
        max_epochs=args.epochs, 
        deterministic=True, 
        gpus=1, 
        progress_bar_refresh_rate=1, 
        callbacks=[EarlyStopping(monitor='train_loss', patience=15), checkpoint_callback, lr_monitor])    
    trainer.fit(model, data_module)    
    return model

In [1]:
# To reload tensorBoard
%load_ext tensorboard

# logs folder path
%tensorboard --logdir=lightning_logs

**Training**



In [47]:
do_training_validation()

Global seed set to 21
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params
----------------------------------
0 | lstm   | LSTM   | 37.6 K
1 | linear | Linear | 306   
----------------------------------
37.9 K    Trainable params
0         Non-trainable params
37.9 K    Total params
0.152     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/sunt/anaconda3/envs/har/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
Global seed set to 21
/home/sunt/anaconda3/envs/har/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

ActionClassificationLSTM(
  (lstm): LSTM(34, 50, num_layers=2, batch_first=True)
  (linear): Linear(in_features=50, out_features=6, bias=True)
)

**Get the saved model**

In [50]:
import os
def get_latest_run_version_ckpt_epoch_no(lightning_logs_dir='lightning_logs', run_version=None):
    if run_version is None:
        run_version = 0
        for dir_name in os.listdir(lightning_logs_dir):
            if 'version' in dir_name:
                if int(dir_name.split('_')[1]) > run_version:
                    run_version = int(dir_name.split('_')[1])                
    checkpoints_dir = os.path.join(lightning_logs_dir, 'version_{}'.format(run_version), 'checkpoints')    
    files = os.listdir(checkpoints_dir)
    ckpt_filename = None
    for file in files:
        print(file)
        if file.endswith('.ckpt'):
            ckpt_filename = file        
    if ckpt_filename is not None:
        ckpt_path = os.path.join(checkpoints_dir, ckpt_filename)
    else:
        print('CKPT file is not present')    
    return ckpt_path

In [51]:
ckpt_path = get_latest_run_version_ckpt_epoch_no()
print('The latest model path: {}'.format(ckpt_path))

epoch=119-step=5399.ckpt
The latest model path: lightning_logs/version_2/checkpoints/epoch=119-step=5399.ckpt
